In [12]:
import npu
import numpy as np
from npu.build.appbuilder import AppBuilder
from npu.build.kernel import Kernel
from npu.utils import OpenCVImageReader, image_plot
from npu.runtime import AppRunner
from PIL import Image
from IPython import get_ipython
import tensorflow as tf

2024-10-08 14:12:49.335850: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-08 14:12:49.336878: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-08 14:12:49.339872: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-08 14:12:49.348359: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-08 14:12:49.361424: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [13]:
kernel_name = "passthrough"
kernel_path = f"./kernels/{kernel_name}.c"
with open(kernel_path, 'r') as file:
    data = file.read()
get_ipython().run_cell_magic('kernel', '', data) 

In [14]:
passthrough.metadata

{'name': 'passthrough_1',
 'tloc': None,
 'ttype': 'CT',
 'ktype': 'passthrough',
 'type': 'CT',
 'ports': {'in_buffer': {'c_dtype': None,
   'direction': None,
   'name': 'in_buffer',
   'ctype': None,
   'shape': None,
   'dtype': None},
  'out_buffer': {'c_dtype': None,
   'direction': None,
   'name': 'out_buffer',
   'ctype': None,
   'shape': None,
   'dtype': None},
  'nbytes': {'c_dtype': 'uint32_t',
   'direction': 'in',
   'name': 'nbytes',
   'ctype': 'rtp',
   'value': 'uint32_t'}}}

In [20]:
passthrough.ports[0].array = np.ndarray([1280, 4], dtype=np.uint8)
passthrough.ports[1].array = np.ndarray([1280, 4], dtype=np.uint8)

In [23]:
passthrough.metadata

{'name': 'passthrough_1',
 'tloc': None,
 'ttype': 'CT',
 'ktype': 'passthrough',
 'type': 'CT',
 'ports': {'in_buffer': {'c_dtype': None,
   'direction': None,
   'name': 'in_buffer',
   'ctype': None,
   'shape': TensorShape([1280, 4]),
   'dtype': 'bfloat16'},
  'out_buffer': {'c_dtype': None,
   'direction': None,
   'name': 'out_buffer',
   'ctype': None,
   'shape': TensorShape([1280, 4]),
   'dtype': 'bfloat16'},
  'nbytes': {'c_dtype': 'uint32_t',
   'direction': 'in',
   'name': 'nbytes',
   'ctype': 'rtp',
   'value': 'uint32_t'}}}

In [24]:
class SimpleApplication(AppBuilder):

    def __init__(self, kernel:Kernel):
        self.kernel = kernel
        super().__init__()

    def callgraph(self, x_in: np.ndarray, x_out: np.ndarray) -> None:
        rows = x_in.shape[0]
        bytes_per_row = x_in.shape[1] * x_in.shape[2]
        for row in range(rows):
            kernel_output = self.kernel(x_in[row], bytes_per_row)
            x_out[row] = kernel_output

app_builder = SimpleApplication(kernel=passthrough)

In [25]:
import numpy as np
input_image = np.zeros(shape=(720,1280,4), dtype=np.uint8)
output_image = np.zeros(shape=(720,1280,4), dtype=np.uint8)

app_builder.build(input_image, output_image)

AttributeError: EagerTensor object has no attribute 'reshape'. 
        If you are looking for numpy-related methods, please run the following:
        tf.experimental.numpy.experimental_enable_numpy_behavior()
      

In [7]:
app = AppRunner('SimpleApplication.xclbin')

In [8]:
img = OpenCVImageReader('images/jpg/ryzenai_future_starts_now.jpg').img

In [9]:
input_image = app.allocate(shape=(720,1280,4), dtype=np.uint8)
output_image = app.allocate(shape=(720,1280,4), dtype=np.uint8)
input_image[:] = img
input_image.sync_to_npu()
app.call(input_image, output_image)
output_image.sync_from_npu()

In [10]:
im = Image.fromarray(output_image).convert('RGB')
im.save("./images/jpg/output_pass.jpg")
print("DONE!")

DONE!


In [1]:
del app

NameError: name 'app' is not defined